In [16]:
import numpy as np
import pandas as pd
import random

def bb84_sim(n=20, eve=False, eve_intercept_prob=1.0):
	# 0 = base recta (↕), 1 = base diagonal (↗)
	bits_alice  = np.random.randint(0, 2, n)
	bases_alice = np.random.randint(0, 2, n)
	bits_eve = None

	if eve:
		bases_eve = np.random.randint(0, 2, n)
		bits_eve  = np.random.randint(0, 2, n)

		for i in range(n):
			if random.random() <= eve_intercept_prob:
				# Eve mide: acierta si su base coincide; si no, obtiene bit aleatorio
				bits_eve[i] = bits_alice[i] if bases_eve[i] == bases_alice[i] else np.random.randint(0, 2)
				# Eve reenvía el fotón con SU resultado y base
				bits_alice[i]  = bits_eve[i]
				bases_alice[i] = bases_eve[i]

	# Bob mide con bases aleatorias
	bases_bob = np.random.randint(0, 2, n)
	bits_bob  = np.empty(n, dtype=int)
	for i in range(n):
		bits_bob[i] = bits_alice[i] if bases_bob[i] == bases_alice[i] else np.random.randint(0, 2)

	# Coincidencia de bases y extracción de la clave
	coincide = bases_alice == bases_bob
	key_bits = bits_bob[coincide]
	key      = ''.join(map(str, key_bits))

	# Historial en DataFrame
	df = pd.DataFrame({
		"Bit_Alice": bits_alice,
		"Base_Alice": np.where(bases_alice == 0, "A", "B"),
		"Base_Bob":   np.where(bases_bob   == 0, "A", "B"),
		"Bit_Bob": bits_bob,
		"Coincide": coincide
	})
	
	if bits_eve is not None:
		df["Bit_Eve"] = bits_eve
		df["Base_Eve"] = bases_eve

	return df, key, key_bits.size

In [17]:
n_fotones = 50
dataframe, clave, longitud = bb84_sim(n=n_fotones, eve=False, eve_intercept_prob=0.5)

print("Historial de la transmisión:")
print(dataframe.to_string(index=False))
print(f"\nTotal de fotones enviados: {n_fotones}")
print(f"Clave final ({longitud} bits): {clave}")
print(f"Rendimiento: {longitud / n_fotones * 100:.2f}%")

Historial de la transmisión:
 Bit_Alice Base_Alice Base_Bob  Bit_Bob  Coincide
         1          B        A        0     False
         1          B        A        0     False
         1          A        B        1     False
         1          A        B        1     False
         0          B        B        0      True
         0          A        B        1     False
         0          A        A        0      True
         1          A        A        1      True
         0          B        B        0      True
         1          B        B        1      True
         0          B        B        0      True
         1          A        B        0     False
         0          B        A        1     False
         0          B        B        0      True
         0          A        B        1     False
         0          A        B        0     False
         0          A        A        0      True
         1          A        A        1      True
         1          B

In [18]:
dataframe, clave, longitud = bb84_sim(n=n_fotones, eve=True, eve_intercept_prob=0.5)

print("Historial de la transmisión:")
print(dataframe.to_string(index=False))
print(f"\nTotal de fotones enviados: {n_fotones}")
print(f"Clave final ({longitud} bits): {clave}")
print(f"Rendimiento: {longitud / n_fotones * 100:.2f}%")

Historial de la transmisión:
 Bit_Alice Base_Alice Base_Bob  Bit_Bob  Coincide  Bit_Eve  Base_Eve
         1          B        A        1     False        1         1
         0          A        B        0     False        1         0
         0          B        B        0      True        0         1
         0          A        B        1     False        1         1
         0          A        B        0     False        0         0
         1          B        A        0     False        1         1
         0          A        B        1     False        0         0
         0          B        A        1     False        0         1
         1          B        B        1      True        1         0
         0          A        A        0      True        0         0
         0          A        B        1     False        0         1
         0          B        B        0      True        0         1
         1          A        A        1      True        1         0
     

# AVG Uninterrupted

In [19]:
N = 1000
n_fotones = 50

total_bits = 0
rendimientos = []

for _ in range(N):
	dataframe, clave, longitud = bb84_sim(n=n_fotones, eve=False, eve_intercept_prob=0.5)
	total_bits += longitud
	rendimientos.append(longitud / n_fotones * 100)

# Statistics
rendimientos_series = pd.Series(rendimientos)

print(f"Estadísticas tras {N} simulaciones con {n_fotones} fotones cada una:")
print(f"Promedio de bits en la clave final: {total_bits / N:.2f}")
print(f"Promedio de rendimiento: {rendimientos_series.mean():.2f}%")
print(f"Desviación estándar del rendimiento: {rendimientos_series.std():.2f}%")
print(f"Máximo rendimiento observado: {rendimientos_series.max():.2f}%")
print(f"Mínimo rendimiento observado: {rendimientos_series.min():.2f}%")

Estadísticas tras 1000 simulaciones con 50 fotones cada una:
Promedio de bits en la clave final: 24.92
Promedio de rendimiento: 49.83%
Desviación estándar del rendimiento: 7.07%
Máximo rendimiento observado: 76.00%
Mínimo rendimiento observado: 28.00%


# AVG Eve Intercept

In [20]:
N = 1000
n_fotones = 50

total_bits = 0
rendimientos = []

for _ in range(N):
	dataframe, clave, longitud = bb84_sim(n=n_fotones, eve=True, eve_intercept_prob=0.5)
	total_bits += longitud
	rendimientos.append(longitud / n_fotones * 100)

# Statistics
rendimientos_series = pd.Series(rendimientos)

print(f"Estadísticas tras {N} simulaciones con {n_fotones} fotones cada una:")
print(f"Promedio de bits en la clave final: {total_bits / N:.2f}")
print(f"Promedio de rendimiento: {rendimientos_series.mean():.2f}%")
print(f"Desviación estándar del rendimiento: {rendimientos_series.std():.2f}%")
print(f"Máximo rendimiento observado: {rendimientos_series.max():.2f}%")
print(f"Mínimo rendimiento observado: {rendimientos_series.min():.2f}%")

Estadísticas tras 1000 simulaciones con 50 fotones cada una:
Promedio de bits en la clave final: 24.92
Promedio de rendimiento: 49.83%
Desviación estándar del rendimiento: 7.21%
Máximo rendimiento observado: 72.00%
Mínimo rendimiento observado: 28.00%
